### Adjust Word Level

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


#### While Loop

In [3]:
lang_folder = "Turkish"
lang_pair = "Arabic"

In [4]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
164294,buldugumuzda,43
164295,boşandıklarını,42
164296,endişenlenme,42
164297,uzaylıymışım,42


In [5]:
#df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence_All.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext.
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [6]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_And_Arabic (ara)_Etymologeek_Result_All.xlsx']

In [7]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,mı,mıknatıs,Turkish,(Noun) Magnet.Magnet.,مِغْنَاطِيس,Arabic (ara),NaN
1,mı,mısdak,Turkish,(Noun) (obsolete) criterion.(obsolete) criterion.,مِصْدَاق,Arabic (ara),NaN
2,şey,şey,Turkish,(Noun Interjection) (slang) genital organ. Thi...,شَيْئاً,Arabic (ara),NaN
3,şey,şeytan,Turkish,(Noun) Devil.Devil.,شَيَّطَ,Arabic (ara),NaN
4,şey,şeytanî,Turkish,(Adjective) Devilish.Devilish.,شَيْطَانِيّ,Arabic (ara),NaN
...,...,...,...,...,...,...,...
362,avra,avrat,Turkish,(Noun) Wife.Wife.,عَوِرَ,Arabic (ara),NaN
363,molla,molla,Turkish,NaN,مَوْلًى,Arabic (ara),NaN
364,ruba,rubai,Turkish,"(Noun) Rubai; quatrain in classical Arabic, Pe...",رباع,Arabic (ara),"Four at a time, in a group of four, in fours."
365,كتاب,كتاب,Turkish,NaN,كتاب,Arabic (ara),"(obsolete). Book. Letter, note, paper, piece o..."


In [8]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [9]:
def word_count_result(df,column_list): # df is dataframe, bigger_than is integer
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list)
    df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [10]:
def word_count_bool(df, word_thresh_num, column_list): # df is a dataframe, bigger_than is an integer
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list)
    df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"count"}, inplace=True)
    
    return  df_word_count["count"][df_word_count.loc[:,"count"] > word_thresh_num].any()  

In [11]:
# while loop code block
twogram_num = 100  # 6*10 min.
twogram_pair_num = 100  # 3*10 min.
word_thresh_num = 7

word_start = 0  # 0
word_end = 28  # 10
step_num = 28  # 10
word_limit = 28  # 200
part_num = 1
while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    ngram_list = []
    for i in df_pair["dict_entry_main"]:
        for j in word_tokenize(i):
            for k in df_word["word"]:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":"twogram_pair"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)

    # twogram
    word_list = df_word["word"].values.tolist()
    data_kind = "twogram"
    twogram_list  = df_twogram_sent.iloc[:,0].values.tolist()
    
    resultlist2 = []

    manager = multiprocessing.Manager()
    resultlist2 = manager.list()
    
    def word_in_wordgroup2(list_var2):
        mergelist = []
        try:
            word = list_var2.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist2.append(list_var2)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup2, twogram_list) # string_word liste 

    result_list2 = list(resultlist2)
    df_result2 = pd.DataFrame(result_list2)
    df_result2 = df_result2.rename(columns = {0: f"{data_kind}"})
    df_result2.iloc[:,0] = df_result2.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
    df_merge2 = pd.merge(df_result2, df_twogram_sent, how="inner", on=f"{data_kind}")
    df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
    df_merge_result2.drop_duplicates(inplace=True)
    df_merge_result2.reset_index(drop=True, inplace=True)
    df_twogram_result = df_merge_result2
    #df_twogram_result = df_twogram_result.head(100)

    # output
    df_output_result = pd.concat([df_word, df_twogram_result, df_lang_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word","twogram","freq_twogram","twogram_pair","freq_twogram_pair"])
    a = 0
    b = 0

    for i in range(0,110):
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert n grams
        try:
            var1 = df_output_result.iloc[a,2]
            freq_var1 = df_output_result.iloc[a,3]
            if (len(df_lesson_result["twogram"]) < twogram_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,"twogram"] = var1
                df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num-1), ["twogram"]): # word count result                
                        a += 1
                        var1 = df_output_result.iloc[a,2]
                        freq_var1 = df_output_result.iloc[a,3]
                        df_lesson_result.loc[i,"twogram"] = var1
                        df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,"twogram"] = np.nan
                    df_lesson_result.loc[i,"freq_twogram"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

        try:
            var2 = df_output_result.iloc[b,4]
            freq_var2 = df_output_result.iloc[b,5]
            if (len(df_lesson_result["twogram_pair"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,"twogram_pair"] = var2
                df_lesson_result.loc[i,"freq_twogram_pair"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num-4), ["twogram_pair"]): # word count result                
                        b += 1
                        var2 = df_output_result.iloc[b,4]
                        freq_var2 = df_output_result.iloc[b,5]
                        df_lesson_result.loc[i,"twogram_pair"] = var2
                        df_lesson_result.loc[i,"freq_twogram_pair"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,"twogram_pair"] = np.nan
                    df_lesson_result.loc[i,"freq_twogram_pair"] = np.nan
            else:
                pass
        except:
            pass
        b += 1

    df_lesson_word_count = word_count_result(df_lesson_result, ["twogram","twogram_pair"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

#### Check Result

In [12]:
df_lesson_result

,word,freq_word,twogram,freq_twogram,twogram_pair,freq_twogram_pair,word_count
0,bir,18835735.0,ne var,62532.0,tamam mı,134129,9.0
1,bu,11062659.0,ben de,59972.0,bir dakika,51838,9.0
2,ne,8025880.0,değil mi,58386.0,bir saniye,26472,9.0
3,ve,7766036.0,ben mi,33652.0,hayır değil,18740,9.0
4,için,5484109.0,çok iyi,33627.0,ne kadar,17952,5.0
...,...,...,...,...,...,...,...
73,NaN,NaN,seni beni,7.0,şarap da,16,NaN
74,NaN,NaN,da mı,6.0,acaba onu,15,NaN
75,NaN,NaN,beni seni,4.0,hayat da,9,NaN
76,NaN,NaN,NaN,NaN,mesela bunu,8,NaN


In [13]:
word_count_result(df_lesson_result, ["twogram"])

,word,word_count
0,ama,6
1,gibi,6
2,çok,6
3,yok,6
4,ve,6
5,var,6
6,sen,6
7,ne,6
8,mı,6
9,mi,6


In [14]:
word_count_result(df_lesson_result, ["twogram_pair"])

,word,word_count
0,acaba,3
1,onu,3
2,evet,3
3,gibi,3
4,iyi,3
...,...,...
72,müsaade,1
73,fıstık,1
74,fikir,1
75,fakat,1


#### Test

In [ ]:
#word_num = 300
#df_word_youtube = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/Turkish/Word Tokenize Merge/Youtube_Word_Tokenize_Merge.csv")
#df_word_select = df_word_youtube.head(word_num)
#df_word_select

In [78]:
df_sent_youtube = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/Turkish/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_sent_youtube

,start_time,end_time,sentence,video_id
0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM
1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM
2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM
3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM
4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM
...,...,...,...,...
387903,00:02:11.720,00:02:14.440,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE
387904,00:02:15.560,00:02:19.320,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE
387905,00:02:20.320,00:02:21.200,sahi,7R3K6EDaBGE
387906,00:02:21.720,00:02:24.480,diğer eşi sende mi hala,7R3K6EDaBGE


In [79]:
df_sent_youtube['start_time'] = pd.to_timedelta(df_sent_youtube['start_time']) # data type converted timedelta for second 
df_sent_youtube['end_time'] = pd.to_timedelta(df_sent_youtube['end_time'])

In [80]:
df_sent_youtube['start_time'] = df_sent_youtube['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_sent_youtube['end_time'] = df_sent_youtube['end_time'].apply(lambda x: x.total_seconds())
df_sent_youtube

,start_time,end_time,sentence,video_id
0,6.68,7.34,sen kimsin peki,JR7nKXxUVtM
1,7.34,8.60,parolayı söyle,JR7nKXxUVtM
2,9.44,11.06,ya ben kim olacağım,JR7nKXxUVtM
3,11.06,11.86,keloğlan ım,JR7nKXxUVtM
4,13.38,13.88,ey,JR7nKXxUVtM
...,...,...,...,...
387903,131.72,134.44,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE
387904,135.56,139.32,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE
387905,140.32,141.20,sahi,7R3K6EDaBGE
387906,141.72,144.48,diğer eşi sende mi hala,7R3K6EDaBGE


In [81]:
#word_list = df_word_select.iloc[:,0].to_list()
word_list = ["her şey yolunda"]

In [82]:
df_word_result = pd.DataFrame()
for i in word_list:
    try:
        word_in_video = df_sent_youtube[df_sent_youtube.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].sample(200) # try and except will try for sample
    except:
        word_in_video = df_sent_youtube[df_sent_youtube.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(200)         
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

,word,start_time,end_time,sentence,video_id
0,her şey yolunda,67.294,72.004,nasıl gidiyor o iyi her şey yolunda,ud9s8aerIJE
1,her şey yolunda,143.287,153.391,herhangi bir şikayetiniz yok değil mi her şey ...,gOq59m1pUi4
2,her şey yolunda,976.820,978.020,her şey yolunda mı,6FtA8_D7Ynw
3,her şey yolunda,332.360,333.780,sevgi her şey yolunda mı,OPfevmWgETE
4,her şey yolunda,315.280,320.149,kısa bir inceleme ile her şey yolunda görünüyo...,_20u1qh4nSk
5,her şey yolunda,144.938,147.559,herhangi bir şikayetimiz yok değil mi her şey ...,BsRzeONLa28
6,her şey yolunda,460.320,462.860,sıkıntı yok gayet iyi her şey yolunda,CNPzcVvZ1nU
7,her şey yolunda,1039.348,1045.208,endişelenme her şey yolunda dedi ve sonra başk...,b39EUtXn88M
8,her şey yolunda,510.080,513.695,sonra her şey yolunda sonra görüşürüz kendine ...,FCStjeSY5R0
9,her şey yolunda,1384.160,1387.380,mamasını da yemiş her şey yolunda dün serum al...,GjlduZjqHxA


In [67]:
df_word_result.start_time = df_word_result.start_time.apply(lambda x: round(x))
df_word_result.end_time = df_word_result.end_time.apply(lambda x: round(x)) 

In [68]:
df_word_result

,word,start_time,end_time,sentence,video_id
0,her şey yolunda,160,163,ekonomi iyi mi her şey yolunda mı yok mu,c9ymkFAiZDk
1,her şey yolunda,145,148,herhangi bir şikayetimiz yok değil mi her şey ...,BsRzeONLa28
2,her şey yolunda,1510,1512,her şey yolunda herhalde kontrol,dWIJFBbJSn4
3,her şey yolunda,1000,1001,her şey yolunda,GhMqYDwawUU
4,her şey yolunda,539,551,genel muayenesine bakıyoruz şimdi her şey yolu...,A4wf93zJZIQ
5,her şey yolunda,143,153,herhangi bir şikayetiniz yok değil mi her şey ...,gOq59m1pUi4
6,her şey yolunda,793,798,hastalık derdi var musibet derdi var her şey y...,vOWbfQ0tIyU
7,her şey yolunda,277,281,onunla her şey yolunda sen böyle yapmaya devam et,uzYXX4RgcO0
8,her şey yolunda,977,978,her şey yolunda mı,6FtA8_D7Ynw
9,her şey yolunda,627,628,anamnezde her şey yolunda,F7OQWEJJR34


In [69]:
df_word_result.start_time.to_list()

[160, 145, 1510, 1000, 539, 143, 793, 277, 977, 627]

In [70]:
df_word_result.end_time.to_list()

[163, 148, 1512, 1001, 551, 153, 798, 281, 978, 628]

In [14]:
df_word_result.video_id.to_list()

['A4wf93zJZIQ',
 'gOq59m1pUi4',
 'ifxg2zkaN5E',
 'yKx4zO8_Jhk',
 'CNPzcVvZ1nU',
 'GWzi144zrEs',
 'GWzi144zrEs',
 '6FtA8_D7Ynw',
 'M5zJ_X8i3rI',
 'WYcGfSiTbYY']

In [71]:
",".join(df_word_result.video_id.to_list())

'c9ymkFAiZDk,BsRzeONLa28,dWIJFBbJSn4,GhMqYDwawUU,A4wf93zJZIQ,gOq59m1pUi4,vOWbfQ0tIyU,uzYXX4RgcO0,6FtA8_D7Ynw,F7OQWEJJR34'

In [4]:
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence_All.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext.
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [5]:
df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [6]:
set_twogram_sent = set(df_twogram_sent.twogram)
set_twogram = set(df_twogram.twogram)

In [7]:
df_ngram = pd.DataFrame(set_twogram.difference(set_twogram_sent))
df_ngram

,0
0,olunca insanlar
1,fikret bilgisayarı
2,oy toplama
3,bunu anlatmalıydın
4,el kitabının
...,...
4171721,eliyle çalışıyor
4171722,daima güvenebileceğim
4171723,pariste yaptırmış
4171724,i̇kâmet belgemi


In [8]:
n_gram_list = df_ngram.iloc[:,0].to_list()
n_gram_list

['olunca insanlar',
 'fikret bilgisayarı',
 'oy toplama',
 'bunu anlatmalıydın',
 'el kitabının',
 've istenmeyen',
 'ey garip',
 'edeceğini şaşırmıştı',
 'peşinde brill',
 'hani size',
 'minseok babasız',
 'evdeki sürtüğe',
 'belicoff bana',
 'olası karın',
 'kilit de',
 'tozu getirdim',
 'mı beklemeyi',
 'uçuş united',
 'konuşabilen bir',
 'lima peruya',
 'xavier slaughtery',
 'çayı burada',
 'isyanlar ve',
 'gözünden kaçmayacak',
 'noktalama işaretiyle',
 '222 ve',
 'zamanında yetişirse',
 'gülerim onlara',
 'güldüren adamları',
 'hediyesini kira',
 'seni tanımadan',
 'çekini kontrol',
 'karı bul',
 'yüzyılın ilişkisiydi',
 'sadece bolonez',
 'ben cinayettenim',
 'bay tamnusı',
 'bozuyordu çünkü',
 'malzeme beklemiyoruz',
 'araba satman',
 'edilmesine kalan',
 'oraya hissettiğim',
 'chrysler fabrikasında',
 'bekleyebilirim diye',
 'adım arkamda',
 'sürücü öğretmenisin',
 'bunu karşılayacak',
 'olduğu kanıtlanamadı',
 'hiromichi horikawa',
 'önce duymamışsınız',
 'istersem istediğim'

In [9]:
df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_n_sent = df_threegram

In [10]:
result_df = []

In [13]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_df = manager.list()

def wordgroup_in_wordgroup(n_gram_list):
    df_wordgroup_result = pd.DataFrame()
    df_result = df_n_sent[df_n_sent.iloc[:,0].str.contains(fr"(^|\s){n_gram_list}(\s|$)")]
    df_result.insert(0,f"all_ngram",n_gram_list)
    df_wordgroup_result = pd.concat([df_wordgroup_result,df_result], axis=0)
    df_wordgroup_result.reset_index(drop=True, inplace=True)
    result_df.append(df_wordgroup_result)    
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(wordgroup_in_wordgroup, n_gram_list) # string_word liste

<ipython-input-13-a99a1acd9821>:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_result = df_n_sent[df_n_sent.iloc[:,0].str.contains(fr"(^|\s){n_gram_list}(\s|$)")]
<ipython-input-13-a99a1acd9821>:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_result = df_n_sent[df_n_sent.iloc[:,0].str.contains(fr"(^|\s){n_gram_list}(\s|$)")]
<ipython-input-13-a99a1acd9821>:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_result = df_n_sent[df_n_sent.iloc[:,0].str.contains(fr"(^|\s){n_gram_list}(\s|$)")]
<ipython-input-13-a99a1acd9821>:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_result = df_n_sent[df_n_sent.iloc[:,0].str.contains(fr"(^|\s){n_gram_list}(\s|$)")]
<ipython-input-13-a99a1acd9821>:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_result = df_n_sent[

In [ ]:
multi_df_sent_list = list(result_df)
df_sent_result = pd.DataFrame()